In [35]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget

# Import loaders (used by viewer and overlays)
from loader import (
    load_cgm_data,
    load_sleep_data,
    load_food_entry_data
)

# Import the CGMViewer class
from viewer import CGMViewer

# Import overlays on CGM (glucose) time-series
from overlays import (
    TimeInRangeOverlay,
    FoodEntryOverlay
)

# Import CGM-based biomarker overlays
from overlays import (
    CgmMeasuresOverlay,
    CvOverlay,
    MageOverlay
)

# Import Multi-modal biomarker overlays
from overlays import (
    WakeupGlucoseOverlay,
    PPGROverlay
)

# Import extensions (e.g. hypnogram) that will be attached in separate panel
from extensions import (
    HypnogramExtension
)

base_path = "./sample_subject"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading Data from the JupyterHealth Exchange

In [36]:
# Initialize the JupyterHealth Client
# See documentation for more detail: https://jupyterhealth.org/software-documentation/
from jupyterhealth_client import JupyterHealthClient, Code
jh_client = JupyterHealthClient()

# Sample study and patient for the purpose of this tutorial
patient_id = 40079
study_id = 30017

# Get basic information about the patient
jh_client.get_patient(patient_id)

{'id': 40079,
 'jheUserId': 10070,
 'identifier': '010101',
 'nameFamily': 'Doe',
 'nameGiven': 'Jane',
 'birthDate': '1998-01-01',
 'telecomPhone': None,
 'telecomEmail': 'janedoejupyter@example.org',
 'organizations': [{'id': 20029,
   'name': 'JupyterCon Tutorial',
   'type': 'other',
   'partOf': 20028,
   'currentUserRole': None,
   'children': []}]}

### Loading CGM Data from JHE

In [38]:
# Using the client
client_cgm_df = jh_client.list_observations_df(
    patient_id = patient_id,
    study_id = study_id,
    code = Code.BLOOD_GLUCOSE.value,
    limit = 10_000
)

cgm_df = load_cgm_data(source="client", client_df=client_cgm_df)
cgm_df.head()

,time,gl
0,2025-01-28 10:21:00+00:00,84.0
1,2025-01-28 10:26:00+00:00,83.0
2,2025-01-28 10:31:00+00:00,81.0
3,2025-01-28 10:36:00+00:00,82.0
4,2025-01-28 10:41:00+00:00,82.0


In [39]:
# You can also load using the local file
cgm_df = load_cgm_data(source="file", base_path=base_path, filename="blood_glucose.json")
cgm_df.head()

,time,gl
0,2025-01-28 10:21:00+00:00,84.0
1,2025-01-28 10:26:00+00:00,83.0
2,2025-01-28 10:31:00+00:00,81.0
3,2025-01-28 10:36:00+00:00,82.0
4,2025-01-28 10:41:00+00:00,82.0


## Loading Sleep and Food Entry Data from JHE

## Using the CGMSandbox

In [3]:
### Initialize the CGMSandbox Viewer and draw daily glucose tracings
viewer = CGMViewer(base_path, filename="blood_glucose.json", gl_range=(0, 250))

### Let's try to add some overlay's to the plain CGM tracing! 
viewer.add_overlay(TimeInRangeOverlay())
viewer.add_overlay(FoodEntryOverlay(base_path, filename="food_entry.json"))

# ### Now, let us extend the plain CGM tracing by adding an hypnogram extension
viewer.add_extensions(HypnogramExtension(base_path, filename="sleep_stage_summary.json"))

# ### Let us use overlay to visualize our CGM-based biomarkers
viewer.add_overlay(CgmMeasuresOverlay())
viewer.add_overlay(CvOverlay())
viewer.add_overlay(MageOverlay(show_ma=True))

# ### We can utilize sleep and diet data to extract other meaninful markers
viewer.add_overlay(WakeupGlucoseOverlay(base_path, filename="sleep_stage_summary.json"))
viewer.add_overlay(PPGROverlay(base_path, filename="food_entry.json"))

viewer.show()